In [11]:
"""
function ClickConnect(){
    console.log("코랩 연결 끊김 방지");
    document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect, 60 * 1000)
"""

'\nfunction ClickConnect(){\n    console.log("코랩 연결 끊김 방지");\n    document.querySelector("colab-toolbar-button#connect").click()\n}\nsetInterval(ClickConnect, 60 * 1000)\n'

##전체 파일 합치기

In [2]:
#----------------------------csv파일 합치기---------------------------------------
# import os
# import pandas as pd

# directory = '/content/drive/MyDrive/2024_1/nlp/fuckingoriginal' #해당 디렉토리에 있는 모든 파일 하나로 합치기
# all_files = []

# for filename in os.listdir(directory):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(directory, filename) # 파일 경로 생성
#         all_files.append(pd.read_csv(file_path))

# combined_df = pd.concat(all_files, ignore_index=True)

# save_filename = 'original_conversation'
# combined_df.to_csv(f'/content/drive/MyDrive/{save_filename}.csv', index=False)

#----------------------------JSON파일 합치기---------------------------------------
import os
import json

directory = '/content/drive/MyDrive/2024_1/nlp/fuckingoriginal'
all_data = []

for filename in os.listdir(directory):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename) # 파일 경로 생성
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            all_data.extend(data)

output_file_path = '/content/drive/MyDrive/original_conversation.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(all_data, file, ensure_ascii=False, indent=4)


##Load Packages##

In [3]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

# for transformers, 최신버전은 에러발생
!pip install transformers==4.35.2
!pip install accelerate==0.24.1

# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

Found existing installation: torch 1.13.1+cu116
Uninstalling torch-1.13.1+cu116:
  Successfully uninstalled torch-1.13.1+cu116
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp310-cp310-linux_x86_64.whl (1977.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
Torch version:1.13.1+cu116
cuda version: 11.6
cudnn version:8302
fatal: destination path 'KoChatGPT' already 

In [4]:
# import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


###(참고) 제공된 코드의 데이터 파일 형식

In [6]:
import json

with open('/content/data_kochatgpt/kochatgpt_1_SFT.jsonl','r') as f:
  ff = json.load(f)
print(ff[1])
print(ff[1]['prompt'])

{'prompt': '쓰던 앱이 유료로 전환됐어', 'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.", 'tokens': 288}
쓰던 앱이 유료로 전환됐어


##Finetune SFT (우린 안해도 될듯)

In [7]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_1_SFT', type=str, default='./content/drive/MyDrive/combined_final_file.csv')
parser.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default='./output_1_SFT')

args = parser.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'  # SK GPT2, https://github.com/SKT-AI/KoGPT2
args.max_epochs = 2

print(args)

Namespace(data_path_1_SFT='./content/drive/MyDrive/combined_final_file.csv', model_name='skt/kogpt2-base-v2', max_epochs=2, train_batch_size=8, output_dir='./output_1_SFT')


In [ ]:
## test & load skt gpt2 kroean
import torch
from transformers import GPT2LMHeadModel, pipeline

from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
print(tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o"))
# ['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id,
                         bos_token_id=tokenizer.bos_token_id,
                         use_cache=True)
generated = tokenizer.decode(gen_ids[0])
# print(generated)


generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)
# generator(
#     ["0 : **는 게임 좋아하니\n1 :",
#     "0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 :",
#     "0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : "],
#     **generation_args
# )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']
근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


[[{'generated_text': '0 : **는 게임 좋아하니\n1 : ***-****\n'}],
 [{'generated_text': '0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 : 아, 저게 진짜 아니야???\n0 : 이거 어떻게 된 거야????\n2 : 그건 뭐지?\n3 : 그냥 찍었잖아..\n3 : 이건 뭐야??!\n3 : 어어?\n3 : 우연찮게 생긴 여고생들 때문에'}],
 [{'generated_text': '0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : 뭘 그런 식으로 얘기하는 거야? 내가 화를 내면서 말했잖아.\n2 : 뭐라고 했어?\n3 : 무슨 일이 있었어?\n4 : 너한테 거짓말을 한 거야?\n5 : 아무 일도 없었어?\n6 : 그건 네 잘못이야!\n7 : 아냐, 이게'}]]

In [8]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
prompt = "null"
PROMPT_DICT = {
    "prompt_input": (
        "욕설이 포함된 아래 입력을 욕설이 없는 예쁜 한국어 문장으로 바꿔줘. 아래 예시를 참고하고 입력 문장과 짧거나 비슷한 길이로 만들어줘\n\n"
        "요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        f"### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": ( #<-이 버전으로 쓸듯
        "욕설이 포함된 아래 입력을 욕설이 없는 예쁜 한국어 문장으로 바꿔줘. 아래 예시를 참고하고 입력 문장과 짧거나 비슷한 길이로 만들어줘\n\nn"
        "input : 우왕 너무 배고파~~\n\n"
        "output : 배고프노 이기야 ~~\n\n"
        "input : 과제 너무 많아서 하기가 싫어\n\n"
        "output : 라도출신 교수게이가 과제를 노무노무 많이내서 하기가 싫노~ .\n\n"
        f"### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

In [9]:
model = AutoModelForCausalLM.from_pretrained(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=512,
)
tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False, single_word

###SFT를 위한 데이터셋 준비

In [10]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):
    '''SFT dataset by wygo'''
    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        ## format
        pattern_instruction = 'prompt'  # 순화를 지시하는 명령 (프롬프트)
        pattern_input = 'input'  # 순화시킬 데이터
        pattern_output = 'completion'  # 순화된 output

        ############################################################
        ## load dataset
        # 내 데이터셋엔 input이 없다 >> KoMo팀은 아니지롱 있지롱
        data_path_1_SFT = '/content/drive/MyDrive/combined_final_file.csv'
        list_data_csv = pd.read_csv(data_path_1_SFT, encoding='utf-8')
        profanity_data = list_data_csv['Profanity']
        cleaned_data = list_data_csv['Cleaned']
        if True:
          print(profanity_data[0])
        # with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
        #     list_data_dict = json.load(json_file)
        #     if True:
        #       print(list_data_dict)

        ############################################################
        ## 데이터셋 만들기, source와 target
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]  # 템플릿 가져오기

        # 입력
        sources = []
        for example in profanity_data:
            if example != "":
                tmp = prompt_no_input.format(example) if isinstance(example, str) else prompt_no_input
            else:
                tmp = prompt_input
            sources.append(tmp)

        # 출력
        targets = []
        for example in cleaned_data:
            targets.append(f"{example}{tokenizer.eos_token}" if isinstance(example, str) else f"{tokenizer.eos_token}")

        if verbose:
            idx = 0
            print(sources[idx])
            print(targets[idx])
            print("Tokenizing inputs... This may take some time...")

        ############################################################
        # data_dict = preprocess(sources, targets, tokenizer)  # https://github.com/Beomi/KoAlpaca/blob/04704348d58b8b1c2e2638d6437a04b4e8ba1823/train.py#L124
        examples = [s + t for s, t in zip(sources, targets)]

        # source data tokenized
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source만
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target


        ## 입력은 source, 출력은 source+target 이지만 학습은 target 부분만
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = IGNORE_INDEX  # source 부분은 -100으로 채운다

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))

    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        """Tokenize a list of strings."""
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )



train_dataset = SFT_dataset(data_path_1_SFT=args.data_path_1_SFT, tokenizer=tokenizer)
eval_dataset  = None  # eval은 안함
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

# check
print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

그냥 게임이나 하자고? ㅋㅋ 니가 게임하는거 보면 암걸릴거같아서 차라리 니 혼자 게임해라 ㅅㅂ 니랑 같이 하기 싫으니까 꺼져


input : tensor([12160, 16122, 20600,  9793,  9241,  9492, 12160, 16122,  9712, 41775,
        34407, 43257, 27729,  8244,   389,  9793,  9182, 11909, 11522,  9038,
        27318, 31618, 10669,  9314, 11451, 25243,  9659,  8244,   375,   375,
          452,  9610,   454, 14197,  9223, 40972, 12371,  9208,  6889,  8615,
          468,   468,   375,   375, 44682,   454, 14197,  9223,  9208,  6889,
         8688,  7119, 13779,  7991, 14380,   468,   375,   375,  9610,   454,
        14197,  9223, 28257, 12371, 27080,  9078,  9495, 16238,   375,   375,
        44682,   454, 14197,  9223,  9755,  7235, 27457, 13643,  6866,  9760,
        37932, 29777,  7119,  7556,  9564,  7071,  7788,  9078,  9495, 15254,
         7119,   468,   739,  9585,   375,   378,   378,   378, 14659, 13394,
        37091, 10651,   383, 25841,  8006, 14914,   375,   452, 36420,   375,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,  6947,  7084, 15403,  9185, 17709,

In [46]:
## 학습 (10min)
# training_args 수정 가능: https://github.com/Beomi/KoAlpaca/blob/main/train.sh 참고
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
training_args = TrainingArguments(
    output_dir="./sft_test", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 3, # Number of update steps between two evaluations.
    save_steps=500, # after # steps model is saved
    warmup_steps=5,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )
trainer = Trainer(
    model=model.to(device),
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=args.output_dir)

cpu


Step,Training Loss


KeyboardInterrupt: 

##Reward Model

In [12]:
# import
import argparse

import loralib as lora
import torch
torch.cuda.empty_cache()
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMRM
from chatgpt.models.gpt import GPTRM
from chatgpt.models.opt import OPTRM
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from datasets import load_dataset
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

import os
import json

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"

# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default='./output_2_RM')
parser.add_argument('--data_path_2_RM', type=str, default='./data_kochatgpt/kochatgpt_2_RM.jsonl', help='https://huggingface.co/datasets/fka/awesome-chatgpt-prompts/blob/main/prompts.csv')
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--dataset', type=str, default='Dahoas/rm-static')
parser.add_argument('--save_path', type=str, default='rm_ckpt.pth')
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=4)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_len', type=int, default=512)  # wygo 추가

args = parser.parse_args(args=[])

# for test
args.max_epochs = 3
args.pretrain = 'skt/kogpt2-base-v2'  # pretrained 모델 가져오기
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

/usr/local/lib/python3.10/dist-packages/torch/library.py:130: UserWarning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::index.Tensor(Tensor self, Tensor?[] indices) -> Tensor
    registered at aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: Meta
  previous kernel: registered at ../aten/src/ATen/functorch/BatchRulesScatterOps.cpp:1053
       new kernel: registered at /dev/null:241 (Triggered internally at ../aten/src/ATen/core/dispatch/OperatorEntry.cpp:150.)
  self.m.impl(name, dispatch_key, fn)


Namespace(output_dir='./output_2_RM', data_path_2_RM='./data_kochatgpt/kochatgpt_2_RM.jsonl', strategy='naive', model='gpt2', pretrain='skt/kogpt2-base-v2', dataset='Dahoas/rm-static', save_path='rm_ckpt.pth', max_epochs=3, batch_size=4, lora_rank=0, max_len=512, verbose=True)


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [35]:
# customizing, https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/nn/gpt_rm.py#L29
from typing import Optional

import torch.nn as nn
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

# from ..base import RewardModel
from chatgpt.models.base import RewardModel


class GPTRM_custom(RewardModel):
    """
    GPT Reward model.
    Args:
        pretrained (str): Pretrained model name or path.
        config (GPT2Config): Model config.
        checkpoint (bool): Enable gradient checkpointing.
        lora_rank (int): Rank of the low-rank approximation.
        lora_train_bias (str): LoRA bias training mode.
    """

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))  # wygo 추가!!!
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()


        # model = model.resize_token_embeddings(len(tokenizer))

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        # 추가, 230421
        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained

    # 추가, 230421, config.json을 생성하기 위해 추가
    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [37]:
# configure model, tokenizer
with strategy.model_init_context():
    # load pretrained gpt2
    if args.model == 'gpt2':
#         tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # tokenizer = AutoTokenizer.from_pretrained(args.pretrain)
        tokenizer = AutoTokenizer.from_pretrained(args.pretrain, padding_side="right", model_max_length=512)
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )
        tokenizer.pad_token = tokenizer.eos_token
        model = GPTRM_custom(pretrained=args.pretrain, lora_rank=args.lora_rank, tokenizer=tokenizer) #.cuda() 붙여서 실행하기

    elif args.model == 'bloom':
        model = BLOOMRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)

    elif args.model == 'opt':
        model = OPTRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

    else:
        raise ValueError(f'Unsupported model "{args.model}"')


    # model.resize_token_embeddings(len(tokenizer))

##Reward Model 데이터 형식

{'prompt': '메뉴판 있어요?',
'completion_0': '죄송합니다, 제가 가상의 챗봇입니다. 제가 메뉴판을 제공할 수는 없습니다. 하지만, 혹시 궁금한 음식이나 음료가 있다면 알려주시면 그에 맞는 정보를 제공해드릴 수 있습니다.',

'completion_1': '이락으로 잘알면 강의할 지 등 공개한 경개는 높아 가고 한다면 객체개를 위한 개서를 한다가 국토법인 경개를 높게 속한 경개는 일 하나 국토법인 경개를 들여 객체개는 더 국토법인 경개를 높이 한다면 객체개를 토법인 경개를 높이 한다고 국토법인 경개를 한다면 국토법인 경개를 높이 토법인 경개는 국토법인 경개를 높이 객체개를 토법인 경개를 한다면 국토법인 경개는다\n\n국토법인 경개를 들여 국토법인 경개를 국토법인 경개를 높이 국토법인 경개를 한다면 국토법인 경개를 한다면 국토법인 토법인 경개를 높이 한다면 국토법인 경개를 높이 한다면 국토법인 경개를 한다면 국토법인 경개를 높이 한다면 국토법인 경개를 한다면 국토법인 경개를 한다',

'completion_2': '네, 여러가지 메뉴가 있습니다. 치킨, 피자, 샐러드, 스파게티, 그리고 다양한 간식과 음료 등이 있습니다.',

'ranking': [0, 2, 1]}

##Transfer KoMo data to RM data format

In [24]:
import random

rank0_path = '/content/drive/MyDrive/combined_emojis_final.csv' #원래문장에 이모지 있는,가장 랭크 높은 데이터
rank1_path = '/content/drive/MyDrive/original_conversation.json' #이모지 없고 문장만 착한 문장인 중간 랭크 데이터
rank2_path = '/content/drive/MyDrive/combined_final_file.csv'#욕이 남발하는 꼴등 데이터

def read_json(path):
  with open(path,'r',encoding='utf-8') as f:
    data = json.load(f)
  return data

def readcsv(path):
  data = pd.read_csv(path)
  return data

def add_ranking_column(data, rank):
    data_with_rank = []
    for index, row in enumerate(data):
        data_dict = {
            'prompt': row,
            'ranking': rank
        }
        data_with_rank.append(data_dict)
    return data_with_rank

rank0_data = readcsv(rank0_path)['Cleaned']
rank1_data = readcsv(rank2_path)['Cleaned'] #일반문장은 욕설 순화 전 문장과 같기 때문에 rank2의 'cleaned'로 처리
rank2_data = readcsv(rank2_path)['Profanity'] #욕설 데이터의 'profanity' 열 값

rank0_data_with_rank = add_ranking_column(rank0_data, 0)
rank1_data_with_rank = add_ranking_column(rank1_data, 1)
rank2_data_with_rank = add_ranking_column(rank2_data, 2)

list_data_dict = []
for i,(r0,r1,r2) in enumerate(zip(rank0_data_with_rank,rank1_data_with_rank,rank2_data_with_rank)):
  combined_lists = [r0,r1,r2]
  random.shuffle(combined_lists)
  temp_dict = {
      'prompt':r1['prompt'],
      'completion_0':combined_lists[0]['prompt'],
      'completion_1':combined_lists[1]['prompt'],
      'completion_2':combined_lists[2]['prompt'],
      'ranking':[combined_lists[0]['ranking'],combined_lists[1]['ranking'],combined_lists[2]['ranking']]
  }
  list_data_dict.append(temp_dict)

In [31]:
total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    # data 1) 0 VS 1
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = str(tmp['completion_0'])
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = str(tmp['completion_1'])
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)


    # data 2) 0 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = str(tmp['completion_0'])
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = str(tmp['completion_2'])
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # data 1) 1 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = str(tmp['completion_1'])
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = str(tmp['completion_2'])
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 20835
after  data num: 62505
data example: 
{'prompt': '겨우 2개? 한판도 못함 진짜', 'chosen': '겨우 2개? 한판도 못함 진짜', 'rejected': '겨우 2개 ㅋㅋㅋ 한판은 무슨 씹덕들이 2개로 오바하네 ㅉㅉ'}


In [32]:
print(total_data_ranking2chosen[45])

{'prompt': '겨우 2개? 한판도 못함 진짜', 'chosen': '겨우 2개? 한판도 못함 진짜', 'rejected': '겨우 2개 ㅋㅋㅋ 한판은 무슨 씹덕들이 2개로 오바하네 ㅉㅉ'}


In [44]:
# prepare for data and dataset
import random
random.seed(230319)
# list_tmp = list(range(10))
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

# train_data = total_data_ranking2chosen[:-1000]  # 29000 학습
# eval_data = total_data_ranking2chosen[-1000:0]  # 1000개만 평가

train_data = total_data_ranking2chosen[:100]  # 29000 학습
eval_data = total_data_ranking2chosen[100:130]  # 1000개만 평가
train_dataset = None
eval_dataset = None
for each in train_data:
  try:
    train_dataset = RewardDataset(each, tokenizer, args.max_len)
  except TypeError:
    continue
for each in eval_data:
  try:
    eval_dataset = RewardDataset(each, tokenizer, args.max_len)
  except TypeError:
      continue

# check
# idx = 3
# print('#'*70)
# print('## prompt ##')
# print(train_data[idx]['prompt'])
# print('#'*70)
# print('## chosen ##')
# print(train_data[idx]['chosen'])
# print('#'*70)
# print('## rejected ##')
# print(train_data[idx]['rejected'])

{'prompt': '나 예전에 위장 안 좋을 때는 그냥 오트밀+따뜻한 물+설탕 해서 먹었는데 이게 정석 아닌가', 'chosen': '이제 낚시 갈 때 썩다배리 차 타고 가야겠어 🎣🚗', 'rejected': '나 예전에 위장 안 좋을 때는 그냥 오트밀+따뜻한 물+설탕 해서 먹었는데 이게 정석 아닌가'}


  0%|          | 0/3 [00:00<?, ?it/s]

######################################################################
## prompt ##
nan
######################################################################
## chosen ##
썬크림이 은근 보호해주나 봄 키키
######################################################################
## rejected ##
nan


In [45]:
# configure optimizer
if args.strategy.startswith('colossalai'):
    optim = HybridAdam(model.parameters(), lr=5e-5)
else:
    optim = Adam(model.parameters(), lr=5e-5)

In [ ]:
# batch_size here is expected to be C(k,2), k means # response of each prompt
# be limited with the format of dataset 'Dahoas/rm-static', we'd better use batch_size as 1
trainer = RewardModelTrainer(model=model,
                             strategy=strategy,
                             optim=optim,
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=args.batch_size,
                             max_epochs=args.max_epochs)

In [ ]:
# train!!
trainer.fit(use_lora=args.lora_rank)

## save
# save model checkpoint after fitting on only rank0
strategy.save_model(model, os.path.join(args.output_dir, 'RM.pt'), only_rank0=True)
# save optimizer checkpoint on all ranks
strategy.save_optimizer(optim,
                        os.path.join(args.output_dir, 'RM_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
                        only_rank0=False)

model.save_pretrained(args.output_dir)  # config.json 생성